In [3]:
import numpy as np
import matplotlib.pyplot as plt
from sktime.libs.vmdpy import VMD

# 时间轴从0到T
T = 1000
fs = 1 / T
t = np.arange(1, T + 1) / T
freqs = 2 * np.pi * (t - 0.5 - fs) / fs

# 各信号成分中心频率
f_1 = 2
f_2 = 24
f_3 = 288

# 构建各模式信号
v_1 = np.cos(2 * np.pi * f_1 * t)
v_2 = 1 / 4 * np.cos(2 * np.pi * f_2 * t)
v_3 = 1 / 16 * np.cos(2 * np.pi * f_3 * t)

# 原始信号，包含三个成分加上随机噪声
f = v_1 + v_2 + v_3 + 0.1 * np.random.randn(v_1.size)

In [4]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import firwin, lfilter, correlate, find_peaks
import pandas as pd
 
 
rows = 1000
time = np.linspace(0, 1, rows)
signal_data =f
# signal_df = pd.DataFrame(signal_data, columns=['Signal'])

In [325]:
#汉宁窗口初始化FIR滤波器组
def initialize_filters(L, K):
    filters = []
    for k in range(1, K+1):
        cutoff = 0.5 / k
        filter = firwin(L, cutoff, window='hann')
        filters.append(filter)
    return filters
#自相关普
def estimate_period(signal):
    correlation = correlate(signal, signal, mode='full')
    correlation = correlation[len(correlation) // 2:]
    peaks, _ = find_peaks(correlation)
    if len(peaks) > 1:
        period = peaks[1]
    else:
        period = len(signal)
    return period

 #FMD函数
def fmd(signal, n, L=30, max_iters=10):
    K = min(10, max(5, n))
    filters = initialize_filters(L, K)
    modes = []
    signal = signal.values.flatten() if isinstance(signal, pd.DataFrame) else signal.flatten()
 
    for i in range(max_iters):
        for filter in filters:
            filtered_signal = lfilter(filter, 1.0, signal)
            period = estimate_period(filtered_signal)
            modes.append(filtered_signal)
 
        if len(modes) >= n:
            break
 
    return modes[:n]

In [143]:
import pandas  as pd
y = np.array(pd.read_csv("data/3号机组抽水态健康样本.csv",encoding="GBK").iloc[:,1])


In [327]:
modes = fmd(y, 2,max_iters=20)

In [6]:
x= y

In [7]:
FilterSize = 30
CutNum = 7
ModeNum = 2
MaxIterNum = 20
fs = 2e4

In [8]:
if FilterSize % 2 == 0:FilterSize += 1 
freq_bound = np.linspace(0, 1 - 1 / CutNum, CutNum)
temp_filters = np.zeros((FilterSize, CutNum))

In [9]:
for n in range(len(freq_bound)):        
        temp_filters[:, n] = firwin(FilterSize, [freq_bound[n] + np.finfo(float).eps, freq_bound[n] + 1 / CutNum - np.finfo(float).eps], window='hann')



In [337]:
temp_filters.shape

(31, 7)

In [11]:
result = pd.DataFrame(
    [[None] * 5 for _ in range(CutNum + 1)],
    columns=['IterCount', 'Iterations', 'CorrMatrix', 'ComparedModeNum', 'StopNum']
)

In [12]:
temp_sig = np.tile(x, (CutNum,1)).T
itercount = 2
iternum = 2
if itercount == 2:iternum = MaxIterNum - (CutNum - ModeNum) * iternum
result[itercount, 0] = iternum

In [16]:
from scipy.signal import hilbert, lfilter

In [ ]:
def CK(x = None,T = None,M = 2): 

    
    x = np.array(x).flatten()
    N = len(x)

    x_shift = np.zeros((M + 1,N))
    x_shift[0,:] = x

    for m in range(M):
        if T < N: x_shift[m + 1,T+1:] = x_shift[m,:-T-1]
    
    # ck = sum(np.prod(x_shift) ** 2) / sum(x ** 2) ** (M + 1)
    numerator = np.sum(np.prod(x_shift, axis=0)**2)
    denominator = np.sum(x**2)**(M + 1)
    ck = numerator / denominator
    return ck

In [91]:
def autocorrelation(data, lag=1):
    data = np.array(data)

    # Compute autocorrelation
    x = data[:-lag]
    y = data[lag:]
    autocorr = np.correlate(x, y, mode='full')
    #return Normalizer().fit_transform(autocorr.reshape(-1,1)).flatten()
    return autocorr

In [93]:
def xcorr(x, y):
    maxlag = np.maximum(np.size(x), np.size(y)) - 1
    c = crosscorr(x, y, maxlag)
    return c
 
 
def crosscorr(x, y, maxlag):
    # Compute cross-correlation for vector inputs. Output is clipped based on
    # maxlag but not padded if maxlag >= max(size(x,1),size(y,1)).
    nx = np.size(x)
    ny = np.size(y)
    m = np.maximum(nx, ny)
    maxlagDefault = m - 1
    mxl = np.maximum(maxlag, maxlagDefault)
 
    m2 = findTransformLength(m)
    X = np.fft.fft(x, m2, 0)
    Y = np.fft.fft(y, m2, 0)
    c1 = np.fft.ifft(X * np.conj(Y), n=None, axis=0)
    # Keep only the lags we want and move negative lags before positive
    # lags.
    c = np.hstack((c1[m2 - mxl + np.arange(mxl)], c1[0:mxl + 1]))
    return c
 
 
def findTransformLength(mm):
    m = 2 * mm
    while True:
        r = m
        for p in [2, 3, 5, 7]:
            while (r > 1) and (r % p == 0):
                r = r / p
        if r == 1:
            break
        m = m + 1
    return m

In [72]:
def find_max_autocorrelation_lag_and_value(data):
    max_lag = len(data)//2
    min_lag=1
    autocorrs = [autocorrelation(data, lag)[0] for lag in range(min_lag, max_lag)]
    max_lag = np.argmax(autocorrs) + min_lag
    max_value = np.max(autocorrs)
    return max_lag, max_value


In [ ]:
def TT(y = None,fs = None): 
    zeroposi = None

    NA = np.correlate(y, y, mode='full')
    NA = NA[int(len(NA) / 2):]

    sample1 = NA[0]
    
    for lag in range(1,len(NA)):
        sample2 = NA[lag]
        if ((sample1 > 0) and (sample2 < 0)):
            zeroposi = lag
            break
        else:
            if ((sample1 == 0) or (sample2 == 0)):
                zeroposi = lag
                break
            else:
                sample1 = sample2
                
    if zeroposi is None:
        return int(len(y)//2)
    NA = NA[zeroposi:]
    max_position = np.argmax(NA)
    T = zeroposi + max_position
    return T

In [121]:
termIter = 30
plotMode = 0
M = 3
n=3
f_init=temp_filters[:,n]
x=temp_sig[:, n]
xxenvelope = np.abs(hilbert(x)) - np.mean(np.abs(hilbert(x)))
T = TT(xxenvelope,fs)
T = np.round(T) 
x = x.flatten()
L = f_init.__len__()
f_final = f_init
N = len(x)
XmT = np.zeros((L, N, M+1))

for m in range(M+1):
    for l in range(L):
        if l == 0:
            XmT[l, (m*T):,m] = x[:N - m*T]
        else:
            XmT[l, 1:,m] = XmT[l-1, :-1,m]

Xinv = np.linalg.inv(XmT[:, :, 0] @ XmT[:, :, 0].T)
f = f_init
ck_best = 0
y = np.zeros(N)
ckIter = []


        

In [138]:
for m in range(M+1):
    for l in range(L):
        if l == 0:
            XmT[l, (m*T):,m] = x[:N - m*T]
        else:
            XmT[l, 1:,m] = XmT[l-1, :-1,m]

In [160]:
print(" T:{}\n N:{}\n M:{}\n L:{}".format(T,N,M,L))

 T:3418
 N:11040
 M:3
 L:31


In [163]:
y

array([nan, nan, nan, ..., nan, nan, nan])

In [161]:
y = (f.T @ XmT[:, :, 0]).T
yt = np.zeros((N, M+1))

for m in range(M+1):
    if m == 0:
        yt[:, m] = y
    else:
        yt[T+1:,m] = yt[:-T-1, m]
alpha = np.zeros((N, M+1))

for m in range(M + 1):
    # 使用 np.prod 计算 yt 在列方向上的乘积，排除第 m+1 列
    # 注意：Python 中索引从 0 开始，因此需要相应地调整
    prod_columns = np.prod(yt[:, np.r_[0:m, m+2:yt.shape[1]]], axis=1)
    # 计算平方并乘以 yt 的第 m+1 列
    alpha[:, m] = (prod_columns ** 2) * yt[:, m]

In [135]:
pd.DataFrame(yt).describe()

,0,1,2,3
count,0.0,11040.0,11040.0,11040.0
mean,NaN,0.0,0.0,0.0
std,NaN,0.0,0.0,0.0
min,NaN,0.0,0.0,0.0
25%,NaN,0.0,0.0,0.0
50%,NaN,0.0,0.0,0.0
75%,NaN,0.0,0.0,0.0
max,NaN,0.0,0.0,0.0


In [124]:


for m in range(M + 1):
    # 使用 np.prod 计算 yt 在列方向上的乘积，排除第 m+1 列
    # 注意：Python 中索引从 0 开始，因此需要相应地调整
    prod_columns = np.prod(yt[:, np.r_[0:m, m+2:yt.shape[1]]], axis=1)
    # 计算平方并乘以 yt 的第 m+1 列
    alpha[:, m] = (prod_columns ** 2) * yt[:, m]

beta = np.prod(yt, axis=1)
Xalpha = np.zeros(L)
for m in range(M):
    Xalpha += XmT[:, :, m] @ alpha[:, m]

f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha
f /= np.sqrt(np.sum(f**2))

ckIter.append(np.sum(np.prod(yt, axis=1)**2) / np.sum(y**2)**(M+1))

if ckIter[-1] > ck_best:
    ck_best = ckIter[-1]

xyenvelope = np.abs(hilbert(y)) - np.mean(np.abs(hilbert(y)))
T = TT(xyenvelope, fs)
T = round(T) if T != None else len(xyenvelope)

XmT = np.zeros((L, N, M+1))

for m in range(M+1):
    for l in range(L):
        if l == 0:
            XmT[l, m*T:,m] = x[:N - m*T]
        else:
            XmT[l, 1:,m] = XmT[l-1, :-1,m]

Xinv = np.linalg.inv(XmT[:, :, 0] @ XmT[:, :, 0].T)

y_final = lfilter(f, 1, x)

C:\Users\26921\AppData\Local\Temp\ipykernel_55560\943637034.py:13: RuntimeWarning: divide by zero encountered in scalar divide
  f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha
C:\Users\26921\AppData\Local\Temp\ipykernel_55560\943637034.py:13: RuntimeWarning: invalid value encountered in matmul
  f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha


In [125]:
T = TT(xxenvelope,fs)
T = round(T) if T != None else len(xyenvelope)
x = x.flatten()
L = f_init.__len__()
f_final = f_init
N = len(x)
XmT = np.zeros((L, N, M+1))

for m in range(M+1):
    for l in range(L):
        if l == 0:
            XmT[l, (m*T):,m] = x[:N - m*T]
        else:
            XmT[l, 1:,m] = XmT[l-1, :-1,m]

Xinv = np.linalg.inv(XmT[:, :, 0] @ XmT[:, :, 0].T)
f = f_init
ck_best = 0
y = np.zeros(N)
ckIter = []

for _ in range(termIter):
        y = (f.T @ XmT[:, :, 0]).T
        yt = np.zeros((N, M+1))
        
        for m in range(M+1):
            if m == 0:
                yt[:, m] = y
            else:
                yt[T:,m] = yt[:-T, m]
        alpha = np.zeros((N, M+1))

        for m in range(M + 1):
            # 使用 np.prod 计算 yt 在列方向上的乘积，排除第 m+1 列
            # 注意：Python 中索引从 0 开始，因此需要相应地调整
            prod_columns = np.prod(yt[:, np.r_[0:m, m+2:yt.shape[1]]], axis=1)
            # 计算平方并乘以 yt 的第 m+1 列
            alpha[:, m] = (prod_columns ** 2) * yt[:, m]

        beta = np.prod(yt, axis=1)
        Xalpha = np.zeros(L)
        for m in range(M):
            Xalpha += XmT[:, :, m] @ alpha[:, m]
        
        f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha
        f /= np.sqrt(np.sum(f**2))

        ckIter.append(np.sum(np.prod(yt, axis=1)**2) / np.sum(y**2)**(M+1))
        
        if ckIter[-1] > ck_best:
            ck_best = ckIter[-1]

        xyenvelope = np.abs(hilbert(y)) - np.mean(np.abs(hilbert(y)))
        T = TT(xyenvelope, fs)
        T = round(T) if T != None else len(xyenvelope)

        XmT = np.zeros((L, N, M+1))

        for m in range(M+1):
            for l in range(L):
                if l == 0:
                    XmT[l, m*T:,m] = x[:N - m*T]
                else:
                    XmT[l, 1:,m] = XmT[l-1, :-1,m]
        
        Xinv = np.linalg.inv(XmT[:, :, 0] @ XmT[:, :, 0].T)
        
        y_final = lfilter(f, 1, x)

C:\Users\26921\AppData\Local\Temp\ipykernel_55560\2803201350.py:45: RuntimeWarning: divide by zero encountered in scalar divide
  f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha
C:\Users\26921\AppData\Local\Temp\ipykernel_55560\2803201350.py:45: RuntimeWarning: invalid value encountered in matmul
  f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha


In [126]:
def xxc_mckd(fs, x, f_init, termIter=None, T=None, M=3, plotMode=0):
    # Set default values for parameters
    if termIter is None:
        termIter = 30
    if plotMode is None:
        plotMode = 0
    if M is None:
        M = 3
    if T is None:
        xxenvelope = abs(hilbert(x)) - np.mean(abs(hilbert(x)))
        T = TT(xxenvelope, fs)

    T = np.round(T)   
    x = x.flatten()
    L = f_init.__len__()
    N = len(x)

    XmT = np.zeros((L, N, M+1))
    for m in range(M+1):
        for l in range(L):
            if l == 0:
                XmT[l, (m*T):,m] = x[:N - m*T]
            else:
                XmT[l, 1:,m] = XmT[l-1, :-1,m]
    
    Xinv = np.linalg.inv(XmT[:, :, 0] @ XmT[:, :, 0].T)
    f = f_init
    ck_best = 0
    y = np.zeros(N)
    ckIter = []

    for n in range(termIter):
        y = (f.T @ XmT[:, :, 0]).T
        yt = np.zeros((N, M))
        
        for m in range(M):
            if m == 0:
                yt[:, m] = y
            else:
                yt[T:] = yt[:-T, m]
        alpha = np.zeros((N, M+1))

        for m in range(M):
            prod_yt = np.prod(yt[:, [i for i in range(M + 1) if i != m]], axis=1) ** 2
            alpha[:, m] = prod_yt * yt[:, m]


        beta = np.prod(yt, axis=1)
        Xalpha = np.zeros(L)
        for m in range(M):
            Xalpha += XmT[:, :, m] @ alpha[:, m]

        f = np.sum(y**2) / (2 * np.sum(beta**2)) * Xinv @ Xalpha
        f /= np.sqrt(np.sum(f**2))

        ckIter.append(np.sum(np.prod(yt, axis=1)**2) / np.sum(y**2)**(M+1))
        
        if ckIter[-1] > ck_best:
            ck_best = ckIter[-1]

        xyenvelope = np.abs(hilbert(y)) - np.mean(np.abs(hilbert(y)))
        T = TT(xyenvelope, fs)
        T = np.round(T)

        XmT = np.zeros((L, N, M+1))
        for m in range(M+1):
            for l in range(L):
                if l == 0:
                    XmT[l, m*T:,m] = x[:N - m*T]
                else:
                    XmT[l, 1:] = XmT[l-1, :-1]
        
        Xinv = np.linalg.inv(XmT[:, :, 0] @ XmT[:, :, 0].T)
        
        y_final = lfilter(f, 1, x)
    
    return y_final, f, ckIter, T

In [ ]:
# from scipy.signal import hilbert, lfilter
# def xxc_mckd(fs, x, f_init, termIter=None, T=None, M=3, plotMode=0):
#     # Set default values for parameters
#     if termIter is None:
#         termIter = 30
#     if plotMode is None:
#         plotMode = 0
#     if M is None:
#         M = 3
#     if T is None:
#         xxenvelope = abs(hilbert(x)) - np.mean(abs(hilbert(x)))

#     T = round(T)   
#     x = x.flatten()
#     L = f_init.__len__()
#     N = len(x)

#     XmT = np.zeros(L,N,M+1)
#     for m in range(M+1):
#         for l in range(0,L):
#             if l ==0:XmT[0,(m*T+1):,m+1] = x[:N-m*T]
#             else: XmT[0,1:,m+1] = XmT[l-1,:-1,m+1]
#     Xinv = np.linalg.inv(np.dot(XmT[:,:,0],XmT[:,:,0].T))

#     f = f_init
#     ck_best = 0
#     y = np.zeros(N,1)
#     b = np.zeros(L,1)
#     ckIter = []
#     n=0
#     f_final = np.array([])

#     while n<=termIter:
#         y = (np.dot(f.T,XmT[:,:,0])).T

#         yt = np.zeros(N,M)
#         f_final[:,1] = f
#         for m in range(M):
#             if m == 0: yt[:,m+1] =y
#             else: yt[T+1:,m+1] = yt[:-T,m]

#         alpha = np.zeros((N, M + 1))

#         for m in range(M+1):
#             prod_yt = np.prod(yt[:, [i for i in range(M + 1) if i != m]], axis=1) ** 2
#             alpha[:, m] = prod_yt * yt[:, m]

#         beta = np.prod(yt, 2)

#         Xalpha = np.zeros(L, 1)

#         for m in range(M+1):
#             Xalpha += XmT[:, :, m] @ alpha[:, m][:, np.newaxis]

#         f= np.sum(y**2)/(2*sum(beta**2))*Xinv @ Xalpha
#         f = f/np.sqrt(sum(f**2))
#         ckIter[n] = sum(np.prod(yt,2)**2)/(sum(y**2)**(M+1))
#         if ckIter>ck_best:ck_best=ckIter[n]

#         #-------------------------------------------------
#         xyenvelope = abs(hilbert(y)) - np.mean(abs(hilbert(y)))
#         T = TT(xyenvelope,fs)
#         T = round(T)
#         T_final = T

#         XmT = np.zeros(L,N,M+1)
#         for m in range(M+1):
#             for l in range(L):
#                 if l == 0 : XmT[0,(m*T+1):]


In [127]:
for n in range(temp_filters.shape[1]):
    f_init = temp_filters[:,n]
    y_Iter, f_Iter, k_Iter, T_Iter = xxc_mckd(fs, temp_sig[:, n], f_init, iternum, None, 1, 0)


IndexError: index 1 is out of bounds for axis 1 with size 1

In [ ]:

while True:
        iternum = 2
        if itercount == 2:
            iternum = MaxIterNum - (CutNum - ModeNum) * iternum

        result[itercount][0] = iternum

        for n in range(temp_filters.shape[1]):
              f_init = temp_filters[:,n]
              y_Iter, f_Iter, k_Iter, T_Iter = xxc_mckd(fs, temp_sig[:, n], f_init, iternum, [], 1, 0)

              result[itercount].append({
                'y_Iter': y_Iter[:, -1],
                'f_Iter': f_Iter[:, -1],
                'k_Iter': k_Iter[:, -1],
                'fft_f_Iter': np.abs(np.fft.fft(f_Iter))[:FilterSize // 2],
                'frequency': np.argmax(np.abs(np.fft.fft(f_Iter))) * (fs / FilterSize),
                'T_Iter': T_Iter
            })

In [63]:
temp_sig

array([[68.87514 , 68.500084, 66.843834, ..., 64.68798 , 65.28982 ,
        64.31443 ]])

In [ ]:
def FMD(fs,x,FilterSize,CutNum,ModeNum,MaxIterNum):
    '''"
    fs: 信号 x 的采样频率。
    x: 待分析的信号。
    FilterSize: 用于分解过程中的滤波器大小。
    CutNum: 将整个频率范围分成的频带数。
    ModeNum: 最终保留的模态数。
    MaxIterNum: 最大迭代次数。

    '''
    #Initialization
    freq_bound = np.arange(0, 1, 1 / CutNum)
    temp_filters = np.zeros((FilterSize, CutNum))

    for n in range(len(freq_bound)):
        eps = np.finfo(float).eps
        b = firwin(FilterSize - 1, [freq_bound[n] + eps, freq_bound[n] + 1 / CutNum - eps])
        temp_filters[:, n] = b


SyntaxError: incomplete input (2313080685.py, line 16)